In [39]:
from operator import add

from typing import TypedDict, Annotated
from pydantic import BaseModel
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-4o")


class AnswerOutput(BaseModel):
    attempt: int
    answer: str


class FeedbackOutput(BaseModel):
    result: bool
    attempt: int
    feedback: str


class State(TypedDict):
    answer: str
    player_answers: Annotated[list[AnswerOutput], add]
    feedbacks: Annotated[list[FeedbackOutput], add]



In [40]:

def player(state: State) -> {"player_answers": list[AnswerOutput]}:
    player_answers = state.get("player_answers", [])
    feedbacks = state.get("feedbacks", [])

    player_llm = llm.with_structured_output(AnswerOutput)

    player_res = player_llm.invoke(
        f"""

        지금 스무고개 게임을 하고 있습니다. 당신은 피드백을 바탕으로 정답을 맞춰야 합니다.

        지금은 {len(player_answers)}째 시도 입니다.

        0번째 시도라면 아무 단어나 제시해주세요.

        이전에 상대방이 준 피드백 을 참고해서 새로운 정답을 제시해 주세요.

        이것은 그동안 답변했던 정답들입니다.
        <player_answers>
        {player_answers}
        </player_answers>

        이것은 그 정답들에 대한 피드백입니다. attempt 값을 가지고 다음 정답을 추론해 주세요.
        </feedbacks>
        {feedbacks}
        </feedbacks>

        답변할 때 이전 시도횟수에서 +1 해서 답변을 해주세요.
        """
    )
    return {
        "player_answers": [{"attempt": player_res.attempt, "answer": player_res.answer}],
    }

def feedback(state: State) -> {"feedbacks" : list[FeedbackOutput]}:
    print("feedBadk🎉")


def gate (state: State):
    player_answers = state.get("player_answers", [])
    last_answer = player_answers[-1]
    print("last_answer", last_answer["answer"], "🔥🔥")

    return last_answer == state["answer"]


In [41]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(State)
(
    graph_builder
    .add_node("player", player)
    .add_node("feedback", feedback)

        .add_edge(START, "player")
    .add_conditional_edges("player", gate, {
        True: END,
        False: "feedback",
    })
)

graph = graph_builder.compile()

In [42]:
graph.invoke({
    "answer" : "사과"
}
)

last_answer 바나나 🔥🔥


{'answer': '사과',
 'player_answers': [{'attempt': 1, 'answer': '바나나'}],
 'feedbacks': []}